In [89]:
import tensorflow as tf

# Load the Keras model
keras_model = tf.keras.models.load_model('bees-wasps.h5')

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /var/folders/22/4rkd_km14n587vm5mk020x800000gq/T/tmprvoc581x/assets


INFO:tensorflow:Assets written to: /var/folders/22/4rkd_km14n587vm5mk020x800000gq/T/tmprvoc581x/assets
2023-11-24 19:02:32.543651: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-24 19:02:32.543662: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-24 19:02:32.543779: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/22/4rkd_km14n587vm5mk020x800000gq/T/tmprvoc581x
2023-11-24 19:02:32.544417: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-24 19:02:32.544422: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/22/4rkd_km14n587vm5mk020x800000gq/T/tmprvoc581x
2023-11-24 19:02:32.546072: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-24 19:02:32.661790: I tensorflow/cc/saved_model/loader.cc:217] Running initialization

In [90]:
# # Load the TFLite model and allocate tensors.
# interpreter = tf.lite.Interpreter(model_path="model.tflite")
# interpreter.allocate_tensors()

# # Get output details
# output_details = interpreter.get_output_details()

# # Print the index of each output
# for output in output_details:
#     print(output['index'])

In [91]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [92]:
img = Image.open('/Users/inristaka/dev/machine-learning-zoomcamp/cohorts/2023/09-serverless/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg')

In [93]:
img = img.resize([150,150], Image.NEAREST)
print(img.size)

(150, 150)


In [94]:
import numpy as np

In [95]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [96]:
X

array([[[[ 0.8901961 ,  0.81960785,  0.7019608 ],
         [ 0.8745098 ,  0.84313726,  0.9529412 ],
         [ 0.827451  ,  0.79607844,  0.9137255 ],
         ...,
         [-0.41960782, -0.3333333 , -0.67058825],
         [-0.30196077, -0.18431371, -0.69411767],
         [-0.40392154, -0.27843136, -0.7647059 ]],

        [[ 0.8980392 ,  0.81960785,  0.7254902 ],
         [ 0.8352941 ,  0.81960785,  0.92156863],
         [ 0.8039216 ,  0.7882353 ,  0.8980392 ],
         ...,
         [-0.45098037, -0.32549018, -0.6784314 ],
         [-0.0745098 ,  0.01176476, -0.60784316],
         [-0.09803921, -0.01176471, -0.6313726 ]],

        [[ 0.85882354,  0.7647059 ,  0.6392157 ],
         [ 0.827451  ,  0.81960785,  0.92941177],
         [ 0.8117647 ,  0.7882353 ,  0.92156863],
         ...,
         [-0.35686272, -0.24705881, -0.64705884],
         [-0.00392157,  0.05882359, -0.5921569 ],
         [-0.1372549 , -0.06666666, -0.5764706 ]],

        ...,

        [[-0.9372549 , -0.8666667 , -0

In [97]:
import tensorflow.lite as tflite

In [98]:
interpreter = tflite.Interpreter(model_path='model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [99]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [104]:
preds

array([[0.14101043]], dtype=float32)

In [105]:
classes = [
    'bees',
    'wasps'
]

dict(zip(classes, preds[0]))

{'bees': 0.14101043}